In [123]:
import pandas as pd
from scipy import stats
import numpy as np
from pandas.api.types import CategoricalDtype
import re

In [124]:
pl0 = pd.read_csv("../Kraftwerksliste_2020.csv", error_bad_lines=False, engine="python", encoding="latin1", sep=";", skiprows=9)
prtr = pd.read_csv("../data/2019-11-25_PRTR-Deutschland_Freisetzungen.csv", error_bad_lines=False, engine="python", encoding="latin1", sep=";", skiprows=0)
#pl0 = pd.read_csv("Kraftwerksliste_CSV.csv", error_bad_lines=False, engine="python")
bm = pd.read_csv("block_plant_mapper.csv", error_bad_lines=False, engine="python")

Skipping line 639: ';' expected after '"'
Skipping line 2278: ';' expected after '"'
Skipping line 2279: ';' expected after '"'
Skipping line 2280: ';' expected after '"'
Skipping line 2281: ';' expected after '"'
Skipping line 2683: ';' expected after '"'
Skipping line 2703: ';' expected after '"'
Skipping line 2704: ';' expected after '"'
Skipping line 3370: ';' expected after '"'
Skipping line 3607: ';' expected after '"'
Skipping line 3608: ';' expected after '"'
Skipping line 3609: ';' expected after '"'
Skipping line 3898: ';' expected after '"'
Skipping line 3902: ';' expected after '"'
Skipping line 4195: ';' expected after '"'
Skipping line 4368: ';' expected after '"'
Skipping line 4391: ';' expected after '"'
Skipping line 4415: ';' expected after '"'
Skipping line 4523: ';' expected after '"'
Skipping line 4605: ';' expected after '"'
Skipping line 4608: ';' expected after '"'
Skipping line 5384: ';' expected after '"'
Skipping line 7470: ';' expected after '"'
Skipping lin

In [125]:
bm['plantid'] = bm['plantid'].apply(lambda x: str(x).replace('/', '_'))

In [126]:
bm.loc[bm.plantid.str.contains("_")]

,plantid,blockid
572,06-04-11_2001178_8_0,BNA0140
573,06-04-11_2001178_8_0,BNA0144
574,06-04-11_2001178_8_1,BNA0139
575,06-04-11_2001178_8_1,BNA0145
576,06-04-11_2001178_8_1,BNA0146
577,06-04-11_2001178_8_2,BNA0141
578,06-04-11_2001178_8_2,BNA1842
579,06-04-11_2001178_8_2,BNA1820
581,06-04-11_2039669_2_0,BNA0147


In [127]:
prtr.describe()

,jahr,geo_lat_wgs84,geo_long_wgs84,haupttaetigkeit,nace_id,jahresfracht_freisetzung,versehentliche_freisetzung,schadstoff_schwellenwert
count,50660.000000,50641.000000,50641.000000,50660.0,50660.000000,5.066000e+04,5.036600e+04,5.063800e+04
mean,2011.942578,51.098703,9.468514,1.0,2630.908133,1.015176e+08,1.275358e+03,8.957339e+06
std,3.163675,1.560206,2.280615,0.0,1296.763830,9.187151e+08,1.148536e+05,2.839353e+07
min,2007.000000,47.547507,6.089509,1.0,123.000000,1.010000e-04,0.000000e+00,1.000000e-04
25%,2009.000000,50.002669,7.416637,1.0,2013.000000,3.907500e+02,0.000000e+00,1.000000e+02
50%,2012.000000,51.378907,8.976796,1.0,3511.000000,2.710000e+04,0.000000e+00,1.000000e+04
75%,2015.000000,52.154765,11.491190,1.0,3700.000000,3.180000e+05,0.000000e+00,1.000000e+05
max,2017.000000,54.813533,15.000934,1.0,9609.000000,3.330000e+10,2.450000e+07,1.000000e+08


In [128]:
pl0.rename(columns={ pl0.columns[10]: "Energieträger_old", pl0.columns[13]: "Energieträger"}, inplace=True)

In [129]:
#pl0

In [130]:
#bm

In [131]:
#pl0.groupby("Energieträger").count()

In [132]:
#bm
pl1 = pl0.loc[np.logical_or(pl0.Energieträger == "Kernenergie", np.logical_or(pl0.Energieträger == "Erdgas", np.logical_or(pl0.Energieträger == "Steinkohle", np.logical_or(pl0.Energieträger == "Braunkohle", pl0.Energieträger == "Mineralölprodukte")))), :]

In [133]:
plt = pl1.dropna(subset=["Kraftwerksnummer Bundesnetzagentur"])

In [134]:
plt = plt.astype({"Energieträger": 'category'})

In [135]:
#plt["Energieträger"] = plt["Energieträger"].astype('category')
#plt["Energieträger"] = plt["Energieträger"].cat.set_categories(["Kernenergie", "Braunkohle", "Steinkohle", "Erdgas", "Mineralölprodukte"], ordered=True)

In [136]:
#plt["Energieträger"] = plt["Energieträger"].astype('category')

In [137]:
#plt.groupby('Energieträger').count()

In [138]:
pl2 = plt.loc[:]

In [139]:
#pl2.groupby("Energieträger").count()

In [140]:
# plt.dtypes

In [141]:
pl2 = plt.rename(columns={ pl1.columns[0]: "blockid", pl1.columns[3]: "PLZ", pl1.columns[4]: "Ort", pl1.columns[8]: "Inbetriebnahme", pl1.columns[9]: "Status", pl1.columns[15]: "KWK", pl1.columns[16]: "Nettoleistung"})
#pl3 = pl2.rename(columns={ pl1.columns[1]: "BlockID" })

In [142]:
def to_year(x):
    if type(x) == type(4):
        return x
    elif type(x) == type(2.5):
        return 0 # handle nans
    return int(x[-4:])

In [143]:
#blocks.loc[blocks.Energieträger == "Kernenergie"]

In [144]:
pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace('.','')
pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace(',','.')
pl2.loc[pl2.blockid == 'BNA0861a', 'Inbetriebnahme'] = 2012
pl2.loc[pl2.blockid == 'BNA1334', 'Inbetriebnahme'] = 2002
pl2.loc[pl2.blockid == 'BNA1141', 'Inbetriebnahme'] = 1970
pl2.loc[pl2.blockid == 'BNA0418', 'Inbetriebnahme'] = 2013
pl2.loc[pl2.blockid == 'BNA1499', 'Inbetriebnahme'] = 1951
pl2.loc[pl2.blockid == 'BNA1502', 'Inbetriebnahme'] = 2013
pl2.loc[pl2.blockid == 'BNA1500', 'Inbetriebnahme'] = 1990
pl2.loc[pl2.blockid == 'BNA1498', 'Inbetriebnahme'] = 1953
pl2.loc[pl2.blockid == 'BNA1260', 'Inbetriebnahme'] = 2013
pl2.loc[pl2.blockid == 'BNA1056', 'Inbetriebnahme'] = 2006


#pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace('\n','')
#pl2['Nettoleistung'] = pl2[pd.to_numeric(pl2["Nettoleistung"])]
#pl2['Nettoleistung'] = pl2['Nettoleistung'].str.replace(';','.')

In [145]:
#pl2[] = pl2[pd.to_numeric(pl2["Nettoleistung"])]
pl2['Inbetriebnahme'] = pl2['Inbetriebnahme'].apply(to_year)
pl2['Nettoleistung'] = pl2['Nettoleistung'].apply(pd.to_numeric)

In [146]:
#bm[pl2.duplicated(['BlockID'], keep=False)].sort_values("BlockID", ascending=False)

In [147]:
#pl2[pl2.duplicated(['blockid'], keep=False)].sort_values("blockid", ascending=False)

In [148]:
cols = [10, 11,12,14,17,18,19]
pl3 = pl2.drop(pl2.columns[cols],axis=1)

In [149]:
#pl3.groupby('Unternehmen').max()

In [150]:
pl4 = pl3.copy()

In [151]:
def fix_company(company):
    #print(company)
    company_dict = {"RWE": "RWE AG", "Vattenfall": "Vattenfall GmbH", "Uniper": "Uniper SE", "EnBW": "EnBW AG", "Steag": "Steag GmbH", "Nordzucker": "Nordzucker AG", "Lausitz Energie": "LEAG"}
    for key, value in company_dict.items():
        if key in str(company):
            return value
    return company

In [152]:
z = "Endgültig Stillgelegt 2015"

In [153]:
def extract_still(x):
    match = re.findall("[0-9]{4}",x)
    return match[0] if match else np.nan

In [154]:
def extract_state(state):
    if state == "In Betrieb":
        return "in Betrieb"
    elif "Endgültig Stillgelegt" in state:
        return "stillgelegt"
    elif "Vorläufig Stillgelegt" in state:
        return "vorläufig stillgelegt"
    else:
        return state

In [155]:
extract_state(z)

'stillgelegt'

In [156]:
def fix_kwk(x):
    return "Nein" if x == "nein" else "Ja" if x == "ja" else x

In [157]:
#pl4 = pl3.ap

In [158]:
pl4['Unternehmen'] = pl4['Unternehmen'].apply(lambda x: fix_company(x))
pl4['Stilllegung'] = pl4['Status'].apply(lambda x: extract_still(x))
pl4['Status'] = pl4['Status'].apply(lambda x: extract_state(x))
pl4['KWK'] = pl4['KWK'].apply(lambda x: fix_kwk(x))

In [159]:
newdf = pl4.copy()

In [160]:
newdf.dtypes

blockid                                         object
Unternehmen                                     object
Kraftwerksname                                  object
PLZ                                             object
Ort                                             object
Straße und Hausnummer (Standort Kraftwerk)      object
Bundesland                                      object
Blockname                                       object
Inbetriebnahme                                   int64
Status                                          object
Energieträger                                 category
KWK                                             object
Nettoleistung                                  float64
Stilllegung                                     object
dtype: object

In [161]:
newdf["Status"] = newdf["Status"].astype('category')
newdf["Status"] = newdf["Status"].cat.set_categories(['in Betrieb', 'Gesetzlich an Stilllegung gehindert', 'Netzreserve',  'Sicherheitsbereitschaft', 'Sonderfall', 'vorläufig stillgelegt', 'stillgelegt'], ordered=True)

In [162]:
newdf["KWK"] = newdf["KWK"].astype('category')
newdf["KWK"] = newdf["KWK"].cat.set_categories(['Ja', 'Nein'], ordered=True)

In [163]:
atest = newdf.groupby("KWK")

In [164]:
newdf.groupby("Energieträger").count()

,blockid,Unternehmen,Kraftwerksname,PLZ,Ort,Straße und Hausnummer (Standort Kraftwerk),Bundesland,Blockname,Inbetriebnahme,Status,KWK,Nettoleistung,Stilllegung
Energieträger,,,,,,,,,,,,,
Braunkohle,77,77,76,77,77,26,77,69,77,77,77,77,17
Erdgas,315,312,304,315,315,172,315,216,315,312,315,315,35
Kernenergie,9,9,9,9,9,2,9,9,9,9,9,9,3
Mineralölprodukte,56,56,56,56,56,30,56,47,56,51,55,56,10
Steinkohle,120,120,119,120,120,75,120,107,120,112,119,120,43


In [165]:
atest.count()

,blockid,Unternehmen,Kraftwerksname,PLZ,Ort,Straße und Hausnummer (Standort Kraftwerk),Bundesland,Blockname,Inbetriebnahme,Status,Energieträger,Nettoleistung,Stilllegung
KWK,,,,,,,,,,,,,
Ja,410,407,397,410,410,243,410,297,410,406,410,410,55
Nein,165,165,165,165,165,61,165,149,165,153,165,165,52


In [166]:
pl3 = newdf

In [167]:
atest = newdf.groupby(["Unternehmen"]).count()
atest.sort_values("Inbetriebnahme", inplace=True, ascending=False)

In [168]:
#print(atest)

In [169]:
atest = pl3.groupby(["Status"]).count()
# atest

In [170]:
# pl3.rename(columns={ pl3.columns[10]: "Energieträger"}, inplace=True)

In [171]:
bm[bm.duplicated(['blockid'], keep=False)].sort_values("blockid", ascending=False)

,plantid,blockid


In [172]:
#bm2 = bm.drop_duplicates()

In [173]:
pl4 = bm.merge(pl3, how="outer", on="blockid")

In [174]:
#pl4[pl4.duplicated(['BlocID'], keep=False)].sort_values("BlockID", ascending=False)

In [175]:
pl5 = pl4[pd.notnull(pl4['blockid'])]
#pl5 = pl4
pl6 = pl5[pd.notnull(pl5['Nettoleistung'])]

In [176]:
# pl6

In [177]:
pl6['plantid'] = pl6['plantid'].apply(lambda x: str(x).replace('/', '_'))

/opt/conda/envs/idp/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [178]:
# pl7 = pl6.replace("/", "_")

In [179]:
#pl7 = pl6.apply(str.replace("/", "_"))

In [180]:
#pl6.loc[pl6.Kraftwerksname == "KW Mittelsbüren"]

In [181]:
pl6['Inbetriebnahme'] = pl6['Inbetriebnahme'].apply(pd.to_numeric)

/opt/conda/envs/idp/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [182]:
blocks = pl6.copy()
stammdaten = pl6.copy()

In [183]:
blocks.sort_values("Inbetriebnahme", inplace=True, ascending=False)
# blocks

In [184]:
plants = blocks.loc[:]
plants = plants.dropna(subset=["plantid"])

In [185]:
#plants_d = plants.loc[plants.duplicated("KraftwerkID", keep=False)]
# plants_d

In [186]:
#plants_c = plants_d.groupby(["KraftwerkID"]).sum()
#plants_count = plants_d.groupby(["KraftwerkID"]).count()

In [187]:
#plants_f = plants_c.reset_index()

In [188]:
#plants_count = plants_count.loc[:, ["blockid"]]

In [189]:
#plants_count = plants_count.reset_index()
# plants_count 

In [190]:
#plants_f = plants_f.merge(plants_count, on="KraftwerkID", how="inner")
# plants_f

In [191]:
#plants[plants.duplicated(['KraftwerkID'], keep=False)].sort_values("BlockID", ascending=False)

In [192]:
plants["Energieträger"] = plants["Energieträger"].astype('category')
plants["Energieträger"] = plants["Energieträger"].cat.set_categories(["Kernenergie", "Braunkohle", "Steinkohle", "Erdgas", "Mineralölprodukte"], ordered=True)
plants["Status"] = plants["Status"].astype('category')
plants["Status"] = plants["Status"].cat.set_categories(['in Betrieb', 'Gesetzlich an Stilllegung gehindert', 'Netzreserve',  'Sicherheitsbereitschaft', 'Sonderfall', 'vorläufig stillgelegt', 'stillgelegt'], ordered=True)
plants["KWK"] = plants["KWK"].astype('category')
plants["KWK"] = plants["KWK"].cat.set_categories(['Ja', 'Nein'], ordered=True)
plants["Bundesland"] = plants["Bundesland"].astype('category')
plants["Bundesland"] = plants["Bundesland"].cat.set_categories(['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen', 'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen', 'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen', 'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen'])

In [247]:
p_grp = plants.groupby("plantid")

In [248]:
plants.groupby("Energip_subgrper").max()

,plantid,blockid,PLZ,Ort,Inbetriebnahme,Status,KWK,Nettoleistung
Energieträger,,,,,,,,
Kernenergie,666-18,BNA0802,97506,Philippsburg,1989.0,stillgelegt,Nein,1410.0
Braunkohle,664-07,BNA1511a,9116,Zülpich,2014.0,stillgelegt,Nein,1060.0
Steinkohle,13-30-1010101,BNA1674,A-44655,Zolling,2015.0,stillgelegt,Nein,875.0
Erdgas,661-99,BNA1947,A-91052,Zülpich,2019.0,stillgelegt,Nein,887.0
Mineralölprodukte,669-01,BNA1338,94469,Zolling,2011.0,stillgelegt,Nein,415.0


In [261]:
p_subgrp = plants.loc[plants['Status'].isin(['in Betrieb','Gesetzlich an Stilllegung gehindert'])].groupby('plantid') # TODO: add 

In [264]:
plants_act = pd.DataFrame()
for plantid, group in p_subgrp:
    entry = {}
    entry["plantid"] = plantid
    entry["activepower"] = group["Nettoleistung"].sum()
    plants_act = plants_act.append(entry, ignore_index=True)

In [267]:
plants_act.sort_values('activepower', ascending=False)

,activepower,plantid
42,3919.0,06-05-100-0248923
55,2812.0,06-05-300-0326774
171,2427.0,14-70-46630660001
58,2361.0,06-05-300-0877384
93,1958.0,06-08-2948214
...,...,...
198,9.7,661-112
59,9.3,06-05-300-0890760
45,4.8,06-05-100-0501870
209,4.3,661-28


In [253]:
#ap = plants.loc[plants.Status == "in Betrieb" | plants.Status == "Gesetzlich an Stilllegung gehindert"]

In [195]:
plants_a = pd.DataFrame()
for plantid, group in p_grp:
    #if not plantid == "06-05-300-9046797":
    #    continue
    entry = {}
    #print(entry)
    entry["plantid"] = plantid
    # entry["BlockID"] = group
    try:
        entry["plantname"] = group["Kraftwerksname"].value_counts().index[0]
    except IndexError:
        entry["plantname"] = np.NaN
    entry["federalstate"] = group["Bundesland"].value_counts().index[0]
    entry["energysource"] = group["Energieträger"].min()
    try:
        entry["chp"] = group["KWK"].min()
    except TypeError:
        #print("aneror")
        #print(plantid)
        #print(group['KWK'].count())
        entry["chp"] = ""
    entry["latestexpanded"] = group["Inbetriebnahme"].max()
    entry["initialop"] = group["Inbetriebnahme"].min()
    entry["totalpower"] = group["Nettoleistung"].sum()
    entry["state"] = group["Status"].min()
    entry["blockcount"] = group["blockid"].count()
    try:
        entry["company"] = group["Unternehmen"].value_counts().index[0]
    except IndexError:
        ;
    #print(entry)
    #break
    plants_a = plants_a.append(entry, ignore_index=True)

In [196]:
int_list = ['blockcount', 'initialop', 'latestexpanded']
for column in int_list:
    plants_a[column] = plants_a[column].astype(int)
#plant['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)
#pl2['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)
#pl2['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)
#pl2['blockcount'] = pl2['blockcount'].apply(pd.to_numeric)

In [197]:
plants_a.dtypes

blockcount          int64
chp                object
company            object
energysource       object
federalstate       object
initialop           int64
latestexpanded      int64
plantid            object
plantname          object
state              object
totalpower        float64
dtype: object

In [198]:
plants_a["energysource"] = plants_a["energysource"].astype('category')
plants_a["energysource"] = plants_a["energysource"].cat.set_categories(["Kernenergie", "Braunkohle", "Steinkohle", "Erdgas", "Mineralölprodukte"], ordered=True)
plants_a["state"] = plants_a["state"].astype('category')
plants_a["state"] = plants_a["state"].cat.set_categories(['in Betrieb', 'Gesetzlich an Stilllegung gehindert', 'Netzreserve',  'Sicherheitsbereitschaft', 'Sonderfall', 'vorläufig stillgelegt', 'stillgelegt'], ordered=True)
plants_a["chp"] = plants_a["chp"].astype('category')
plants_a["chp"] = plants_a["chp"].cat.set_categories(['Ja', 'Nein'], ordered=True)
plants_a["federalstate"] = plants_a["federalstate"].astype('category')
plants_a["federalstate"] = plants_a["federalstate"].cat.set_categories(['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen', 'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen', 'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen', 'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen'])

In [199]:
plants_a.dtypes

blockcount           int64
chp               category
company             object
energysource      category
federalstate      category
initialop            int64
latestexpanded       int64
plantid             object
plantname           object
state             category
totalpower         float64
dtype: object

In [200]:
plants_a.loc[plants_a.plantid == "06-05-300-0326774"]

,blockcount,chp,company,energysource,federalstate,initialop,latestexpanded,plantid,plantname,state,totalpower
71,9,Ja,RWE AG,Braunkohle,Nordrhein-Westfalen,1963,2002,06-05-300-0326774,Niederaußem,in Betrieb,3656.0


In [201]:
plants_a.loc[plants_a.plantid == "06-00176010435"]

,blockcount,chp,company,energysource,federalstate,initialop,latestexpanded,plantid,plantname,state,totalpower
33,4,Ja,Uniper SE,Steinkohle,Hessen,1965,1992,06-00176010435,Staudinger,in Betrieb,1624.0


In [202]:
plants_a.loc[plants_a.plantid == "06-05-900-0865327"]

,blockcount,chp,company,energysource,federalstate,initialop,latestexpanded,plantid,plantname,state,totalpower
105,9,Nein,RWE AG,Steinkohle,Nordrhein-Westfalen,1973,1984,06-05-900-0865327,Gersteinwerk,in Betrieb,2011.0


In [203]:
#plants_a = plants.drop_duplicates(subset="KraftwerkID")

In [204]:
column_titles = ['plantid', 'plantname', 'federalstate','energysource', 'chp', 'latestexpanded', 'initialop', 'totalpower', 'state', 'blockcount', 'company']
plants_b = plants_a.reindex(columns=column_titles)

In [205]:
#plants_b

In [206]:
blocks.drop(['Ort', 'PLZ', 'Straße und Hausnummer (Standort Kraftwerk)'], axis=1, inplace=True)

In [207]:
blocks[blocks.duplicated(['blockid'], keep=False)].sort_values("blockid", ascending=False)

,plantid,blockid,Unternehmen,Kraftwerksname,Bundesland,Blockname,Inbetriebnahme,Status,Energieträger,KWK,Nettoleistung,Stilllegung
519,06-11-01-1105604,BNA0085b,Vattenfall GmbH,Moabit,Berlin,Moabit GT 7,1971.0,in Betrieb,Mineralölprodukte,Nein,34.0,NaN
520,06-11-01-1105604,BNA0085b,Vattenfall GmbH,Moabit,Berlin,Moabit GT 7,1971.0,stillgelegt,Mineralölprodukte,Nein,17.0,2018
534,06-11-01-1105608,BNA0083,Vattenfall GmbH,Wilmersdorf,Berlin,Wilmersdorf GT2 und GT3,1977.0,in Betrieb,Mineralölprodukte,Ja,184.0,NaN
535,06-11-01-1105608,BNA0083,Vattenfall GmbH,Wilmersdorf,Berlin,Wilmersdorf GT1,1977.0,stillgelegt,Mineralölprodukte,Ja,92.0,2018


In [208]:
#blocks

In [209]:
plants_b.sort_values("initialop", ascending=False, inplace=True)

In [210]:
#drop_list = ['Blockname', 'KWK', 'Nettoleistung', 'Stilllegung', 'Straße und Hausnummer (Standort Kraftwerk)', 'PLZ', 'Ort', 'Nettoleistung']
#plants_g = plants_f.drop(drop_list, axis=1)

In [211]:
#plants_h = plants_g.rename(columns={"KraftwerkID": "plantid", "Gesamtleistung": "totalpower", "Blockzahl": "blockcount", "BlockID": "blockid", "Bundesland": "federalstate", "Energieträger": "energysource", "Inbetriebnahme": "latestexpanded", "Kraftwerksname": "plantname", "Status": "state", "Unternehmen": "company"})

In [212]:
plants_final = plants_b.loc[:]

In [213]:
plants_final.dtypes

plantid             object
plantname           object
federalstate      category
energysource      category
chp               category
latestexpanded       int64
initialop            int64
totalpower         float64
state             category
blockcount           int64
company             object
dtype: object

In [214]:
# plants_final

In [215]:
# stammdaten

In [216]:
# cols = [0,2,3,9,10,11,12]
stammdaten = pl6.copy()
drop_list = ['plantid', 'Blockname', 'Energieträger', 'Inbetriebnahme', 'KWK', 'Kraftwerksname', 'Nettoleistung', 'Status', 'Stilllegung', 'Unternehmen']
stammdaten.drop(drop_list, axis=1, inplace=True)

In [217]:
stmp = pl6.copy()
stammdaten2 = stmp.merge(prtr, how="left", left_on="plantid", right_on="kennnummer")

In [218]:
#stammdaten.to_csv("stammdaten.csv", index=False)

In [219]:
#plants_final.to_csv("plants.csv", index=False)

In [220]:
#blocks.to_csv("blocks_3.csv", index=False)

In [221]:
#stammdaten2

In [222]:
DROP_ST = ['blockid', 'jahr', 'kennnummer','betriebsname','betriebsname_2','plz','ort','strasse','hausnr','bundesland','flusseinzugsgebiet','taet_nr','taetigkeit','activity','haupttaetigkeit','branche','sector','nace_id','nace_wirtschaftszweig','nace_sector','stoffgruppe','substances_group','schadstoff','pollutant','umweltkompartiment','releases_to','jahresfracht_freisetzung','versehentliche_freisetzung','schadstoff_schwellenwert','einheit','unit','bestimmungsmethode','determination_method','schutzgrund_fracht','confidential_reason_release','schutzgrund_betrieb','confidential_reason_facility']
DROP_ST2 = ['jahr', 'kennnummer','betriebsname','betriebsname_2','plz','ort','strasse','hausnr','bundesland','flusseinzugsgebiet','taet_nr','taetigkeit','activity','haupttaetigkeit','branche','sector','nace_id','nace_wirtschaftszweig','nace_sector','stoffgruppe','substances_group','schadstoff','pollutant','umweltkompartiment','releases_to','jahresfracht_freisetzung','versehentliche_freisetzung','schadstoff_schwellenwert','einheit','unit','bestimmungsmethode','determination_method','schutzgrund_fracht','confidential_reason_release','schutzgrund_betrieb','confidential_reason_facility']
DROP_ST3 = ['jahr', 'kennnummer', 'bundesland', 'flusseinzugsgebiet', "betriebsname", "betriebsname_2", 'taet_nr','taetigkeit','activity','haupttaetigkeit','branche','sector','nace_id','nace_wirtschaftszweig','nace_sector','stoffgruppe','substances_group','schadstoff','pollutant','umweltkompartiment','releases_to','jahresfracht_freisetzung','versehentliche_freisetzung','schadstoff_schwellenwert','einheit','unit','bestimmungsmethode','determination_method','schutzgrund_fracht','confidential_reason_release','schutzgrund_betrieb','confidential_reason_facility']


In [223]:
#DROP_ST = ['jahr']

In [224]:
plants_d1 = plants_final.merge(prtr, how="left", left_on="plantid", right_on="kennnummer")

In [225]:
#plants_d1.dtypes

In [226]:
pld2 = plants_d1.drop_duplicates(['plantid'])

In [268]:
pld3 = pld2.drop(DROP_ST3, axis=1)
plants_final1 = pld3

In [269]:
plants_final2 = plants_final1.merge(plants_act, on="plantid", how='left')

In [270]:
plants_final2

,plantid,plantname,federalstate,energysource,chp,latestexpanded,initialop,totalpower,state,blockcount,company,plz,ort,strasse,hausnr,geo_lat_wgs84,geo_long_wgs84,activepower
0,06-09-162-0025-0001,Werk 1.1,Bayern,Erdgas,Ja,2019,2019,27.1,in Betrieb,2,BMW AG,80788,München,Lerchenauer Str.,76,48.179175,11.561502,27.1
1,661-10,Küstenkraftwerk K.I.E.L.,Schleswig-Holstein,Erdgas,Ja,2019,2019,188.0,in Betrieb,1,Stadtwerke Kiel AG,NaN,NaN,NaN,NaN,NaN,NaN,188.0
2,661-31,KWK Landshut,Bayern,Erdgas,Ja,2019,2019,17.6,in Betrieb,1,BMW AG,NaN,NaN,NaN,NaN,NaN,NaN,17.6
3,661-93,HKW Dresden-Nord,Sachsen,Erdgas,Ja,2018,2018,11.5,in Betrieb,1,DREWAG Stadtwerke Dresden GmbH,NaN,NaN,NaN,NaN,NaN,NaN,11.5
4,661-107,HKW Tinz,Thüringen,Erdgas,Ja,2018,2018,22.1,in Betrieb,1,ENGIE Deutschland GmbH,NaN,NaN,NaN,NaN,NaN,NaN,22.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,662-20,Spitzenkraftwerk,Bayern,Mineralölprodukte,Ja,0,0,19.0,stillgelegt,1,Eurowatt Spitzenkraft GmbH,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320,14326,GuD-Ikw Staßfurt,Sachsen-Anhalt,Erdgas,Ja,2015,0,109.0,in Betrieb,2,CIECH Energy Deutschland GmbH,39418,Staßfurt,An der Löderburger Bahn,4a,51.863617,11.573377,100.0
321,661-92,GuD Anlage Spreetal,Sachsen,Erdgas,Nein,0,0,50.0,stillgelegt,1,Alpiq Spreetal GmbH,NaN,NaN,NaN,NaN,NaN,NaN,NaN
322,662-01,Brunsbüttel,Schleswig-Holstein,Mineralölprodukte,Nein,0,0,254.0,stillgelegt,4,Vattenfall GmbH,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [229]:
#stammdaten2.dtypes

In [230]:
stammdaten3 = stammdaten2.drop(DROP_ST, axis=1)

In [231]:
stammdaten4 = stammdaten3.drop_duplicates(['plantid'])

In [232]:
#list(stammdaten2)

In [233]:
'''


"CREATE TABLE blocks(plantid TEXT, blockid TEXT NOT NULL PRIMARY KEY, blockdescription TEXT, federalstate TEXT, energysource TEXT, initialop INTEGER, chp TEXT,
blockname TEXT, netpower REAL, state TEXT, endop TEXT, company TEXT, FOREIGN KEY (blockid) REFERENCES addresses(blockid) ON DELETE CASCADE);"
662-01|BNA0164|Vattenfall GmbH|Brunsbüttel|Schleswig-Holstein|GT D|0.0|stillgelegt|Mineralölprodukte|Nein|63.5|201

'''


'\n\n\n"CREATE TABLE blocks(plantid TEXT, blockid TEXT NOT NULL PRIMARY KEY, blockdescription TEXT, federalstate TEXT, energysource TEXT, initialop INTEGER, chp TEXT,\nblockname TEXT, netpower REAL, state TEXT, endop TEXT, company TEXT, FOREIGN KEY (blockid) REFERENCES addresses(blockid) ON DELETE CASCADE);"\n662-01|BNA0164|Vattenfall GmbH|Brunsbüttel|Schleswig-Holstein|GT D|0.0|stillgelegt|Mineralölprodukte|Nein|63.5|201\n\n'

In [234]:
blocks2 = blocks[['plantid', 'blockid', 'Blockname', 'Bundesland', 'Energieträger', 'Inbetriebnahme', 'KWK', 'Kraftwerksname', 'Nettoleistung', 'Status', 'Stilllegung', 'Unternehmen']]

In [235]:
list(blocks2)

['plantid',
 'blockid',
 'Blockname',
 'Bundesland',
 'Energieträger',
 'Inbetriebnahme',
 'KWK',
 'Kraftwerksname',
 'Nettoleistung',
 'Status',
 'Stilllegung',
 'Unternehmen']

In [271]:
stammdaten.to_csv("stammdaten_nh.csv", index=False, header=False)
blocks2.to_csv("blocks_nh.csv", index=False, header=False)
plants_final2.to_csv("plants_nh.csv", index=False, header=False)
stammdaten.to_csv("stammdaten.csv", index=False)
blocks2.to_csv("blocks.csv", index=False)
plants_final2.to_csv("plants.csv", index=False)

In [237]:
stammdaten

,blockid,PLZ,Ort,Straße und Hausnummer (Standort Kraftwerk),Bundesland
0,BNA1401a,41517,Grevenbroich,Energiestraße 101,Nordrhein-Westfalen
1,BNA1401b,41517,Grevenbroich,Energiestraße 101,Nordrhein-Westfalen
2,BNA0115,4007,Böhlen,Werkstraße,Sachsen
3,BNA0116,4564,Böhlen,Werkstraße,Sachsen
4,BNA0793,32469,Petershagen,NaN,Nordrhein-Westfalen
...,...,...,...,...,...
578,BNA0146,28237,Bremen,Otavistr. 7-9,Bremen
579,BNA0141,28237,Bremen,Auf den Delben 35,Bremen
581,BNA1820,28237,Bremen,NaN,Bremen
582,BNA1334,28237,Bremen,Waterbergstraße 14,Bremen


In [238]:
#bm2.to_csv("bpm.csv", index=False)

In [239]:
#blocks

In [240]:
blocks.groupby('Energieträger').count()

,plantid,blockid,Unternehmen,Kraftwerksname,Bundesland,Blockname,Inbetriebnahme,Status,KWK,Nettoleistung,Stilllegung
Energieträger,,,,,,,,,,,
Braunkohle,77,77,77,76,77,69,77,77,77,77,17
Erdgas,315,315,312,304,315,216,315,312,315,315,35
Kernenergie,9,9,9,9,9,9,9,9,9,9,3
Mineralölprodukte,56,56,56,56,56,47,56,51,55,56,10
Steinkohle,120,120,120,119,120,107,120,112,119,120,43


In [241]:
pl4 = blocks.loc[blocks['Energieträger'] == "Mineralölprodukte"]

In [242]:
#pl4

In [243]:
#pd.set_option('display.max_rows', None)

In [244]:
#pl4.loc[pl4['Energieträger'] == "Mineralölprodukte"].sort_values(["Bundesland", "Unternehmen", "Kraftwerksname"])

In [245]:
#plants_final2